In [1]:
import os
import json
from datetime import datetime, timedelta
import pandas as pd

def edit_short_memory(agent_folder, edit_date, wakeup_time="07:00"):
    with open(os.path.join(agent_folder, "short_memory_cache.json")) as f:
        ori_short_memory = json.load(f)

    last_date = datetime.strptime(edit_date, "%m-%d-%Y") - timedelta(days=1)
    last_date_data = pd.read_csv(os.path.join(agent_folder, f"activity_hist/{last_date.strftime('%m-%d-%Y')}.csv"))
    start_time = f"{edit_date} 00:00"
    end_time = f"{edit_date} {wakeup_time}"

    ori_short_memory.keys()
    ori_short_memory["cur_date"] = edit_date
    ori_short_memory["cur_time"] = "00:00"
    ori_short_memory["cur_activity"] = "Sleep"

    ori_short_memory["schedule"] = {'0':{
        "start_time" : start_time,
        "end_time" : end_time,
        "event": "Sleep"
    }}
    ori_short_memory["cur_event"] = {
        "start_time" : start_time,
        "end_time" : end_time,
        "event": "Sleep"
    }
    ori_short_memory["cur_event_index"] = 0
    ori_short_memory["cur_decompose"] = [{
        'activity': 'Sleep',
        'start_time': start_time,
        'end_time': end_time
    }]
    ori_short_memory["cur_decompose_index"] = 0

    ori_short_memory["cur_location_list"] = [{
        'start_time': start_time,
        'end_time': end_time,
        'location': last_date_data['location'].iloc[-1],
        'longitude': last_date_data['longitude'].iloc[-1],
        'latitude': last_date_data['latitude'].iloc[-1]
    }]
    ori_short_memory["cur_location_list_index"] = 0

    ori_short_memory["cur_heartrate_list"] = [{
        'mean': last_date_data["heartrate"].tail(30).mean(),
        'std': last_date_data["heartrate"].tail(30).std(),
        'start_time': start_time,
        'end_time': end_time
    }]
    ori_short_memory["cur_heartrate_list_index"] = 0

    ori_short_memory["cur_chatbot_dict"] = {}

    ori_short_memory["cache"] = ""


    with open(os.path.join(agent_folder, "short_memory_cache.json"), "w") as f:
        json.dump(ori_short_memory, f)

In [ ]:
edit_short_memory(
    agent_folder=".Users/ba20c6a3-4736-3840-9d73-3093ff6b3885/agents/1",
    edit_date="03-02-2024",
    wakeup_time="08:00"
)